In [ ]:
# default_exp projects.robustness_benchmark

# Robustness Benchmark
> Benchmark utility code

In [ ]:
# hide
from nbdev.showdoc import *
from fastcore.test import *

%load_ext autoreload
%autoreload 2

In [ ]:
# export
import os
import torch
import pandas as pd
import numpy as np


from enum import Enum 
from collections import Counter
from functools import partial
from scp.analysis.binary import performance
from scp.utils.dict import apply_to_vals, flatten_intra

## Constants

In [ ]:
# export
Corruption = Enum("Corruption", 
                  "gaussian_noise, shot_noise, impulse_noise,\
                   defocus_blur, motion_blur, zoom_blur,\
                   black_corner, char,\
                   brightness_up, brightness_down, contrast, elastic_transform, pixelate, jpeg_compression,\
                   speckle_noise, gaussian_blur, bubble, saturate,\
                   sanity_check")

Perturbation = Enum("Perturbation",
                    "gaussian_noise, shot_noise,\
                     motion_blur, zoom_blur,\
                     char\
                     brightness, translate, rotate, tilt, scale,\
                     speckle_noise, gaussian_blur, bubble, shear,\
                     sanity_check")

max_severity = 6 # severity ranges from 1 to 5 inclusive
num_frames = 31 # number of perturbation frames
classes = ["nevus", "melanoma"] # diagnostic classes

# AlexNet baseline (unadjusted o.c. and averaged over 5 identical runs)
# requires python >= 3.9
class BenchmarkBaseline():
    '''AlexNet baseline'''
    
    # SAM
    sam = pd.DataFrame({'clean': 0.20590927835051548}, index=[0])
    
    # SAM-C
    samc = pd.DataFrame({'gaussian_noise'   : 0.4031661855670104,
             'shot_noise'       : 0.3797690721649484,
             'impulse_noise'    : 0.4142465979381443,
             'defocus_blur'     : 0.26223587628865985,
             'motion_blur'      : 0.2529162886597938,
             'zoom_blur'        : 0.35642556701030925,
             'black_corner'     : 0.2864305154639175,
             'char'             : 0.21900288659793812,
             'brightness_up'    : 0.2397558762886598,
             'brightness_down'  : 0.32653938144329897,
             'contrast'         : 0.4829649484536082,
             'elastic_transform': 0.21677113402061854,
             'pixelate'         : 0.21391587628865974,
             'jpeg_compression' : 0.2977707216494845}, index=[0]) 
    
    # SAM-C-Extra
    samce = pd.DataFrame({'speckle_noise': 0.3187744329896908,
              'gaussian_blur': 0.2505723711340206,
              'bubble'       : 0.401682474226804,
              'saturate'     : 0.335340206185567}, index=[0])
    
    # SAM-P
    samp = pd.DataFrame({'gaussian_noise': 0.3045558989048004,
             'shot_noise'    : 0.2891536056995392,
             'motion_blur'   : 0.010909090749919416,
             'zoom_blur'     : 0.018934169411659242,
             'char'          : 0.023134795948863033,
             'brightness'    : 0.01862068921327591,
             'translate'     : 0.023343783244490626,
             'rotate'        : 0.04443051218986512,
             'tilt'          : 0.03381400182843208,
             'scale'         : 0.04390804693102837}, index=[0])
    
    # SAM-P-Extra
    sampe =  pd.DataFrame({'speckle_noise': 0.2331452488899231,
               'gaussian_blur': 0.005015673954039813,
               'bubble'       : 0.02865203730762005,
               'shear'        : 0.038390804082155235}, index=[0])
    


In [ ]:
BenchmarkBaseline.sam

,clean
0,0.205909


## Analysis

In [ ]:
# export
# get balanced error rate
def bal_error_rate(v):
    perf = performance(v[1], v[0][:,1], bal_err_rate=True)
    return perf["bal_err_rate"]

# get normal error rate
def error_rate(v):
    perf = performance(v[1], v[0][:,1], err_rate=True)
    return perf["err_rate"]

# get auroc
def auroc(v):
    perf = performance(v[1], v[0][:,1], auroc=True)
    return perf["auroc"]

# get acc
def acc(v):
    perf = performance(v[1], v[0][:,1])
    return perf["acc"]

# get spec
def spec(v):
    perf = performance(v[1], v[0][:,1])
    return perf["sens"]

# get sens
def sens(v):
    perf = performance(v[1], v[0][:,1])
    return perf["spec"]

# get sens
def bal_acc(v):
    perf = performance(v[1], v[0][:,1], bal_acc=True)
    return perf["bal_acc"]

def flip_rate(k, v, n):
    preds = v[0]
    stop = preds.shape[0]
    assert stop%n == 0, f"Number of predictions ({stop}) is not evenly divisible by stepsize ({n})!"
    
    noise_seq = False
    if "noise" in k:
        noise_seq = True
        
    # calculate flip rate for a single image sequence (which contains 'n' number of frames)
    n_sum, m = 0, 0
    for i in range(0, stop, n):
        m += 1
        seq_preds = preds[i:i+n].argmax(dim=1)
        
        if noise_seq:
            n_sum += sum(seq_preds[0]!=seq_preds[1:])
        else:
            n_sum += sum(seq_preds[:-1]!=seq_preds[1:])
    
    return (n_sum/(m*(n-1))).item()

In [ ]:
# export

# get flib probability
def flip_prob(k, v, n):
    preds = v[0]
    stop = preds.shape[0]
    assert stop%n == 0, f"Number of predictions ({stop}) is not evenly divisible by stepsize ({n})!"
    
    noise_seq = False
    if "noise" in k:
        noise_seq = True
        
    # calculate flip rate for a single image sequence (which contains 'n' number of frames)
    n_sum, m = 0, 0
    for i in range(0, stop, n):
        m += 1
        seq_preds = preds[i:i+n].argmax(dim=1)
        
        if noise_seq:
            n_sum += sum(seq_preds[0]!=seq_preds[1:])
        else:
            n_sum += sum(seq_preds[:-1]!=seq_preds[1:])
    
    return (n_sum/(m*(n-1))).item()

def relative_perf(perf_c:dict, perf_cl:float):
    
    def subtract(v, x): return v-x
    
    apply_subtract = partial(subtract, x=perf_cl)
    return apply_to_vals(perf_c, apply_subtract)

def adjust_by_baseline(perf:dict, base:dict):
    
    perf = perf.copy()
    for tfm in perf.keys():
        perf[tfm] = perf[tfm]/base[tfm]
        
    return perf

In [ ]:
# export
def score_sam(df:pd.DataFrame, eval_func):
    '''Compute performance score for SAM using 'eval_func'
    
    Parameters
    ----------
    df : pd.DataFrame
        dataframe with columns "nevus" and "melanoma" which contain probability scores 
        and column "ground_truth" which contains a 0 (nevus) or 1 (melanoma)
        
    eval_func : callable
        function which is used to evaluate the predictions 
        
    Returns
    -------
    perf : dict
        Stores performance score 
    '''
    perf = dict()
    preds = (torch.Tensor(np.array(df[["nevus", "melanoma"]])), torch.LongTensor(np.array(df["ground_truth"])))
    perf["clean"] = eval_func(preds)
    
    return perf

def score_samc(df:pd.DataFrame, eval_func):
    '''Compute performance score for SAM-C/SAM-C-Extra using 'eval_func'
    
    Parameters
    ----------
    df : pd.DataFrame
        dataframe with columns "nevus" and "melanoma" which contain probability scores 
        and column "ground_truth" which contains a 0 (nevus) or 1 (melanoma)
        
    eval_func : callable
        function which is used to evaluate the predictions 
        
    Returns
    -------
    perf : dict
        Stores performance score 
    '''
    # add transformation type to df (required for scoring corruptions and perturbations)
    df["tfms"] = df.image_path.apply(lambda x: x.split("/")[2])
    
    # add severity levels to df (only needed for corruptions)
    df["severities"] = df.image_path.apply(lambda x: x.split("/")[3])
    assert df["severities"].nunique()==5, f"Invalid number of severities ({df['severities'].nunique()})"
    
    perf = dict()
    for (k1, k2), group_df in df.groupby(["tfms", "severities"]):
        preds = (torch.Tensor(np.array(group_df[["nevus", "melanoma"]])), torch.LongTensor(np.array(group_df["ground_truth"])))

        if perf.get(k1): 
            perf[k1][k2] = eval_func(preds)
        else:
            perf[k1] = dict()
            perf[k1][k2] = eval_func(preds)

    perf = apply_to_vals(flatten_intra(perf), np.mean)
    
    return perf

def score_samp(df:pd.DataFrame, eval_func):
    '''Compute performance score for SAM-P using 'eval_func'
    
    Parameters
    ----------
    df : pd.DataFrame
        dataframe with columns "nevus" and "melanoma" which contain probability scores 
        and column "ground_truth" which contains a 0 (nevus) or 1 (melanoma)
        
    eval_func : callable
        function which is used to evaluate the predictions 
        
    Returns
    -------
    perf : dict
        Stores performance score 
    '''
    # add transformation type to df (required for scoring corruptions and perturbations)
    df["tfms"] = df.image_path.apply(lambda x: x.split("/")[2])
    
    perf = dict()
    
    for k1, group_df in df.groupby(["tfms"]):
        preds = (torch.Tensor(np.array(group_df[["nevus", "melanoma"]])), torch.LongTensor(np.array(group_df["ground_truth"])))
        perf[k1] = eval_func(k1, preds)
        
    return perf

def save_perf(df:pd.DataFrame, out_path:str, out_file:str):
    if not os.path.exists(out_path):
        os.mkdir(out_path)
    df.to_csv(f"{out_path}/{out_file}.csv", index=False)